In [22]:
import pandas as pd
from utils import load_filtered_data

In [23]:
city = 'Sicklerville'
name = 'users&bgt'

# load city data
data = load_filtered_data(city)

In [24]:
# creating the nodes

nodes = data['user']

# rename user_id column to id
# rename name (column name that cant be used because of igraph) to people_name
nodes = nodes.rename(columns={'user_id': 'id', 'name': 'people_name'})

nodes

,id,people_name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,RgDVC3ZUBqpEe6Y1kPhIpw,Monica,1282,2009-02-23 14:08:31,12640,10005,11149,"2009,2010,2011,2012,2013,2014,2015,2016","sHozd2pcOKwHtPr8VlZJfg, 7mL9cvICl8fuCQTM89a-SA...",804,...,102,108,138,65,898,2584,1731,1731,551,177
1,zkamNMEjihh3zN7lC7_WVw,Matthew,15,2006-02-04 15:53:46,35,4,22,NaN,"YLzb3jjc3p-XAOBRggcrgA, 9lcq01JUhRY2yW8AxU3TIA...",1,...,1,0,0,0,1,3,2,2,0,0
2,gVFxZMcuG_Tal2_TnpmUPg,Stephanie,146,2008-05-05 00:16:33,347,100,124,"2011,2012,2013,2014,2015,2016","uUh_q6rVMhKxwt0eaiphRg, jRqWx2z7jdSzpvqqswqNnw...",6,...,13,1,0,0,10,8,21,21,8,0
3,g0_x4kVvJAYuk96oCcbOmw,Michael,455,2009-04-15 12:46:06,1823,696,663,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2018","_BcWyKQL16ndpBdggh2kNA, mx50KP6rXgutH36fhnL6Vw...",46,...,11,0,1,5,39,33,57,57,32,8
4,-NbeVN5tnwdyYAvdNkKMjw,Dominic,1101,2012-04-25 19:31:00,6704,1959,4179,"2012,2013,2014,2015,2016,2017,2018,2019,20,20,...","9IRuYmy5YmhtNQ6ei1p-uQ, 6NvkQATK0J2nmEWsuzhHig...",193,...,33,18,2,2,312,457,637,637,137,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073,CUE13op55l9WZp6fBTUWnA,Nicole,2,2016-08-19 13:24:47,0,0,0,NaN,None,0,...,0,0,0,0,0,0,0,0,0,0
3074,cACn6Qc1owO7hV_ZhcP74Q,Allison,1,2019-11-10 00:05:23,0,0,0,NaN,None,0,...,0,0,0,0,0,0,0,0,0,0
3075,SpTYEhfInvALG1le1uu21w,Ray,2,2016-01-18 16:55:04,5,0,0,NaN,None,0,...,0,0,0,0,0,0,0,0,0,0
3076,sGOCaCP7_SMlaRKPNO__CQ,Tahjae,2,2019-05-04 13:07:23,0,0,0,NaN,None,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# creating the edges

temp_edges_struct = {'from': [], 'to': [], 'weight': [], 'type': []}
connected = {}

for business in data['review']['business_id'].unique():
    reviews_of_business = data['review'][data['review']['business_id'] == business]
    users_that_reviewed_business = reviews_of_business['user_id'].unique()
    for i in range(len(users_that_reviewed_business)-1):
        for j in range(i+1, len(users_that_reviewed_business)):
            if users_that_reviewed_business[i] in connected:
                connected[users_that_reviewed_business[i]].append(users_that_reviewed_business[j])
            else:
                connected[users_that_reviewed_business[i]] = [users_that_reviewed_business[j]]
            if users_that_reviewed_business[j] in connected:
                connected[users_that_reviewed_business[j]].append(users_that_reviewed_business[i])
            else:
                connected[users_that_reviewed_business[j]] = [users_that_reviewed_business[i]]

for business in data['review']['business_id'].unique():
    reviews_of_business = data['review'][data['review']['business_id'] == business]
    users_that_reviewed_business = reviews_of_business['user_id'].unique()
    for i in range(len(users_that_reviewed_business)-1):
        for j in range(i+1, len(users_that_reviewed_business)):
            # from i to j
            temp_edges_struct['from'].append(users_that_reviewed_business[i])
            temp_edges_struct['to'].append(users_that_reviewed_business[j])
            temp_edges_struct['weight'].append(connected[users_that_reviewed_business[i]].count(users_that_reviewed_business[j]))
            temp_edges_struct['type'].append('reviewed-same-restaurant')
            # from j to i
            temp_edges_struct['from'].append(users_that_reviewed_business[j])
            temp_edges_struct['to'].append(users_that_reviewed_business[i])
            temp_edges_struct['weight'].append(connected[users_that_reviewed_business[j]].count(users_that_reviewed_business[i]))
            temp_edges_struct['type'].append('reviewed-same-restaurant')

edges = pd.DataFrame(temp_edges_struct)

# remove repeated lines
edges = edges.drop_duplicates()

edges

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,from,to,weight,type
0,1lm9t9D8Sh_YlF69ER0U4w,yuh6GcXMMOPe-ZlcRGs6dA,1,reviewed-same-restaurant
1,yuh6GcXMMOPe-ZlcRGs6dA,1lm9t9D8Sh_YlF69ER0U4w,1,reviewed-same-restaurant
2,1lm9t9D8Sh_YlF69ER0U4w,sQVfy7VzAW5PVVhq1yYB2w,1,reviewed-same-restaurant
3,sQVfy7VzAW5PVVhq1yYB2w,1lm9t9D8Sh_YlF69ER0U4w,1,reviewed-same-restaurant
4,1lm9t9D8Sh_YlF69ER0U4w,varJ-fgCyGMwGz9xtMCYrA,2,reviewed-same-restaurant
...,...,...,...,...
279427,9w7YZBHzbt0mVrvO_ThiZA,7TUCJHMkn-2DZdRyZeJSDg,1,reviewed-same-restaurant
279428,7TUCJHMkn-2DZdRyZeJSDg,eou_N8D5TBvS3d9-tgNf2g,1,reviewed-same-restaurant
279429,eou_N8D5TBvS3d9-tgNf2g,7TUCJHMkn-2DZdRyZeJSDg,1,reviewed-same-restaurant
279430,9w7YZBHzbt0mVrvO_ThiZA,eou_N8D5TBvS3d9-tgNf2g,1,reviewed-same-restaurant


In [26]:
# save data
nodes.to_csv(f'nodes_and_edges/{city}_{name}_nodes.csv', index=False)
edges.to_csv(f'nodes_and_edges/{city}_{name}_edges.csv', index=False)